# Preprocessing Utah Site Specific data for WaDEQA upload.
- Date Updated: 12/03/2021
- Purpose:  To preprocess the UDWRi and UDWRe data into one master file for simple DataFrame creation and extraction

Notes:
- Will treat UDWRi System + UDWRe data as POUs, and UDWRi Source data as PODs.  Will tie together with "CommunityWaterSupplySystem" WaDE field.
- For annual data, assume start = 01/01 & end =  12/31 for now.
- Seperate out water use System data data by customer type / benefical use (e.g. Domestic, Commerical, Industrial, Insitutional, Total).
- Seperate out the water use Source data by monthly and again by annual.
- create missing elements (water source type).

In [ ]:
# Needed Libararies
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [ ]:
# Working Directory and Input File
workingDir = "G:/Shared drives/WaDE Data/Utah/SS_PublicSupplyWaterUse/RawInputData"

os.chdir(workingDir)

## UDWRi System (POU) Data

In [ ]:
# Dataframe Creation - system Data
sys_Input = "UDWRi_SystemData_input.csv"
df_sys = pd.read_csv(sys_Input, encoding = "ISO-8859-1")
df_sys.head(1)

In [ ]:
# System Data - Domestic
df_sys_Domestic = pd.DataFrame(index=df_sys.index)

# Variable Info
df_sys_Domestic['in_VariableCV'] = "Delivered Water Use"
df_sys_Domestic['in_VariableSpecificCV'] = "Delivered Water Use_Annual_Domestic"

# SiteVariableAmounts_fact Info
df_sys_Domestic['linkKey'] = df_sys['System ID']
df_sys_Domestic['in_CommunityWaterSupplySystem'] = df_sys['System Name']
df_sys_Domestic['in_Amount'] = df_sys['Domestic Use']
df_sys_Domestic['in_PopulationServed'] = df_sys['Domestic Connections']
df_sys_Domestic['in_BenUse'] = "Domestic"
df_sys_Domestic['in_CustomerTypeCV'] = "Domestic"
df_sys_Domestic['in_ReportYearCV'] = df_sys['History Year']
df_sys_Domestic['in_TimeframeEnd'] = '12/31/' + df_sys['History Year'].astype(str)
df_sys_Domestic['in_TimeframeStart'] = '01/01/' + df_sys['History Year'].astype(str)

# Water Source Info
df_sys_Domestic['in_WaterSourceTypeCV'] = "Unspecified"

print(len(df_sys_Domestic))
df_sys_Domestic.head(1)

In [ ]:
# System Data - Commercial
df_sys_Commercial = pd.DataFrame(index=df_sys.index)

# Variable Info
df_sys_Commercial['in_VariableCV'] = "Delivered Water Use"
df_sys_Commercial['in_VariableSpecificCV'] = "Delivered Water Use_Annual_Commercial"

# SiteVariableAmounts_fact Info
df_sys_Commercial['linkKey'] = df_sys['System ID']
df_sys_Commercial['in_CommunityWaterSupplySystem'] = df_sys['System Name']
df_sys_Commercial['in_Amount'] = df_sys['Commercial Use']
df_sys_Commercial['in_PopulationServed'] = df_sys['Commercial Connections']
df_sys_Commercial['in_BenUse'] = "Commercial"
df_sys_Commercial['in_CustomerTypeCV'] = "Commercial"
df_sys_Commercial['in_ReportYearCV'] = df_sys['History Year']
df_sys_Commercial['in_TimeframeEnd'] = '12/31/' + df_sys['History Year'].astype(str)
df_sys_Commercial['in_TimeframeStart'] = '01/01/' + df_sys['History Year'].astype(str)

# Water Source Info
df_sys_Commercial['in_WaterSourceTypeCV'] = "Unspecified"

print(len(df_sys_Commercial))
df_sys_Commercial.head(1)

In [ ]:
# System Data - Industrial
df_sys_Industrial = pd.DataFrame(index=df_sys.index)

# Variable Info
df_sys_Industrial['in_VariableCV'] = "Delivered Water Use"
df_sys_Industrial['in_VariableSpecificCV'] = "Delivered Water Use_Annual_Industrial"

# SiteVariableAmounts_fact Info
df_sys_Industrial['linkKey'] = df_sys['System ID']
df_sys_Industrial['in_CommunityWaterSupplySystem'] = df_sys['System Name']
df_sys_Industrial['in_Amount'] = df_sys['Industrial Use']
df_sys_Industrial['in_PopulationServed'] = df_sys['Industrial Connections']
df_sys_Industrial['in_BenUse'] = "Industrial"
df_sys_Industrial['in_CustomerTypeCV'] = "Industrial"
df_sys_Industrial['in_ReportYearCV'] = df_sys['History Year']
df_sys_Industrial['in_TimeframeEnd'] = '12/31/' + df_sys['History Year'].astype(str)
df_sys_Industrial['in_TimeframeStart'] = '01/01/' + df_sys['History Year'].astype(str)

# Water Source Info
df_sys_Industrial['in_WaterSourceTypeCV'] = "Unspecified"

print(len(df_sys_Industrial))
df_sys_Industrial.head(1)

In [ ]:
# System Data - Institutional
df_sys_Institutional = pd.DataFrame(index=df_sys.index)

# Variable Info
df_sys_Institutional['in_VariableCV'] = "Delivered Water Use"
df_sys_Institutional['in_VariableSpecificCV'] = "Delivered Water Use_Annual_Institutional"

# SiteVariableAmounts_fact Info
df_sys_Institutional['linkKey'] = df_sys['System ID']
df_sys_Institutional['in_CommunityWaterSupplySystem'] = df_sys['System Name']
df_sys_Institutional['in_Amount'] = df_sys['Institutional Use']
df_sys_Institutional['in_PopulationServed'] = df_sys['Institutional Connections']
df_sys_Institutional['in_BenUse'] = "Institutional"
df_sys_Institutional['in_CustomerTypeCV'] = "Institutional"
df_sys_Institutional['in_ReportYearCV'] = df_sys['History Year']
df_sys_Institutional['in_TimeframeEnd'] = '12/31/' + df_sys['History Year'].astype(str)
df_sys_Institutional['in_TimeframeStart'] = '01/01/' + df_sys['History Year'].astype(str)

# Water Source Info
df_sys_Institutional['in_WaterSourceTypeCV'] = "Unspecified"

print(len(df_sys_Institutional))
df_sys_Institutional.head(1)

In [ ]:
# System Data - DCII
df_sys_DCII = pd.DataFrame(index=df_sys.index)

# Variable Info
df_sys_DCII['in_VariableCV'] = "Delivered Water Use"
df_sys_DCII['in_VariableSpecificCV'] = "Delivered Water Use_Annual_DCII"

# SiteVariableAmounts_fact Info
df_sys_DCII['linkKey'] = df_sys['System ID']
df_sys_DCII['in_CommunityWaterSupplySystem'] = df_sys['System Name']
df_sys_DCII['in_Amount'] = df_sys['Total Use']
df_sys_DCII['in_PopulationServed'] = df_sys['Total Connections']
df_sys_DCII['in_BenUse'] = "DCII"
df_sys_DCII['in_CustomerTypeCV'] = "DCII"
df_sys_DCII['in_ReportYearCV'] = df_sys['History Year']
df_sys_DCII['in_TimeframeEnd'] = '12/31/' + df_sys['History Year'].astype(str)
df_sys_DCII['in_TimeframeStart'] = '01/01/' + df_sys['History Year'].astype(str)

# Water Source Info
df_sys_DCII['in_WaterSourceTypeCV'] = "Unspecified"

print(len(df_sys_DCII))
df_sys_DCII.head(1)

In [ ]:
# Concatenate System Data into one long dataframe.
frames = [df_sys_Domestic, df_sys_Commercial, df_sys_Industrial, df_sys_Institutional, df_sys_DCII]
df_sys_all = pd.concat(frames)

df_sys_all = df_sys_all.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(df_sys_all))
df_sys_all.head(1)

#### UDWRe Culinary Service Area Data
- Need to tie the UDWRi system data to the UDWRe site info

In [ ]:
# Dataframe Creation - UDWRe site information
csa_Input = "UDWRe_CulinaryWaterServiceAreas_input.csv"
df_csa = pd.read_csv(csa_Input, encoding = "ISO-8859-1")
df_csa.head(1)

In [ ]:
# Output dataframe for Culinary Service Area Data

df_csaOut = pd.DataFrame(index=df_csa.index)
df_csaOut['linkKey'] = df_csa['WRID']

# Site Info
df_csaOut['in_CoordinateMethodCV'] = "Centroid of Area"
df_csaOut['in_Latitude'] = df_csa['Latitude'].astype(float)
df_csaOut['in_Longitude'] = df_csa['Longitude'].astype(float)
df_csaOut['in_PODorPOUSite'] = "POU"
df_csaOut['in_SiteName'] = df_csa['WRENAME']
df_csaOut['in_SiteNativeID'] = "POU" + df_csa['WRID'].astype(str)
df_csaOut['in_SiteTypeCV'] = "Unspecified"

df_csaOut = df_csaOut.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(df_csaOut))
df_csaOut.head(1)

In [ ]:
# Create output dataframe.  Merge the two dataframes into one.
df_sys_out = pd.merge(df_sys_all, df_csaOut, left_on='linkKey', right_on='linkKey', how='left')

df_sys_out = df_sys_out.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(df_sys_out))
df_sys_out

## UDWRi Source (POD) Data

In [ ]:
# Dataframe Creation - system Data
sour_Input = "UDWRi_SourceData_input.csv"
df_sour = pd.read_csv(sour_Input, encoding = "ISO-8859-1")

# drop rows with a null value
df_sour = df_sour.dropna(subset=['Year'])

# Adjust data type of fields
df_sour['Year'] = df_sour['Year'].astype(int)

print(len(df_sour))
df_sour.head(1)

In [ ]:
# Creating Water Source Type.
wsTypeDict = {
    "Well" : "Groundwater",
    "Well/Spring" : "Groundwater",
    "Well Field" : "Groundwater",
    "Well/Stream" : "Groundwater",
    "Tunnel" : "Groundwater",
    "Stream" : "Surface Water",
    "Spring" : "Surface Water",
    "Reservoir" : "Surface Water",
    "Drain" : "Surface Water",
    "Lake" : "Surface Water"}
def assignWaterSourceTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()
        try:
            outList = wsTypeDict[String1]
        except:
            outList = "Unspecified"
    return outList

df_sour['in_WaterSourceTypeCV'] = df_sour.apply(lambda row: assignWaterSourceTypeCV(row['Source Type']), axis=1)
df_sour['in_WaterSourceTypeCV'].unique()

In [ ]:
# Source Data - Annual
df_sour_Ann = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Ann['in_VariableCV'] = "Withdrawal"
df_sour_Ann['in_VariableSpecificCV'] = "Withdrawal_Annual_DCII"

# SiteVariableAmounts_fact Info
df_sour_Ann['linkKey'] = ""
df_sour_Ann['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Ann['in_Amount'] = df_sour['Total']
df_sour_Ann['in_PopulationServed'] = ""
df_sour_Ann['in_BenUse'] = "DCII"
df_sour_Ann['in_CustomerTypeCV'] = "DCII"
df_sour_Ann['in_ReportYearCV'] = df_sour['Year']
df_sour_Ann['in_TimeframeEnd'] = '12/31/' + df_sour['Year'].astype(str)
df_sour_Ann['in_TimeframeStart'] = '01/01/' + df_sour['Year'].astype(str)

# Water Source Info
df_sour_Ann['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Ann['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Ann['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Ann['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Ann['in_PODorPOUSite'] = "POD"
df_sour_Ann['in_SiteName'] = df_sour['Source Name']
df_sour_Ann['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Ann['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Ann))
df_sour_Ann.head(1)

In [ ]:
# Source Data - Monthly Jan
df_sour_Jan = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Jan['in_VariableCV'] = "Withdrawal"
df_sour_Jan['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Jan['linkKey'] = ""
df_sour_Jan['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Jan['in_Amount'] = df_sour['Jan'] # <- change here
df_sour_Jan['in_PopulationServed'] = ""
df_sour_Jan['in_BenUse'] = "DCII"
df_sour_Jan['in_CustomerTypeCV'] = "DCII"
df_sour_Jan['in_ReportYearCV'] = df_sour['Year']
df_sour_Jan['in_TimeframeEnd'] = '01/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Jan['in_TimeframeStart'] = '01/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Jan['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Jan['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Jan['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Jan['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Jan['in_PODorPOUSite'] = "POD"
df_sour_Jan['in_SiteName'] = df_sour['Source Name']
df_sour_Jan['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Jan['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Jan))
df_sour_Jan.head(1)

In [ ]:
# Source Data - Monthly Feb
df_sour_Feb = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Feb['in_VariableCV'] = "Withdrawal"
df_sour_Feb['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Feb['linkKey'] = ""
df_sour_Feb['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Feb['in_Amount'] = df_sour['Feb'] # <- change here
df_sour_Feb['in_PopulationServed'] = ""
df_sour_Feb['in_BenUse'] = "DCII"
df_sour_Feb['in_CustomerTypeCV'] = "DCII"
df_sour_Feb['in_ReportYearCV'] = df_sour['Year']
df_sour_Feb['in_TimeframeEnd'] = '02/28/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Feb['in_TimeframeStart'] = '02/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Feb['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Feb['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Feb['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Feb['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Feb['in_PODorPOUSite'] = "POD"
df_sour_Feb['in_SiteName'] = df_sour['Source Name']
df_sour_Feb['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Feb['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Feb))
df_sour_Feb.head(1)

In [ ]:
# Source Data - Monthly Mar
df_sour_Mar = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Mar['in_VariableCV'] = "Withdrawal"
df_sour_Mar['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Mar['linkKey'] = ""
df_sour_Mar['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Mar['in_Amount'] = df_sour['Mar'] # <- change here
df_sour_Mar['in_PopulationServed'] = ""
df_sour_Mar['in_BenUse'] = "DCII"
df_sour_Mar['in_CustomerTypeCV'] = "DCII"
df_sour_Mar['in_ReportYearCV'] = df_sour['Year']
df_sour_Mar['in_TimeframeEnd'] = '03/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Mar['in_TimeframeStart'] = '03/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Mar['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Mar['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Mar['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Mar['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Mar['in_PODorPOUSite'] = "POD"
df_sour_Mar['in_SiteName'] = df_sour['Source Name']
df_sour_Mar['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Mar['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Mar))
df_sour_Mar.head(1)

In [ ]:
# Source Data - Monthly Apr
df_sour_Apr = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Apr['in_VariableCV'] = "Withdrawal"
df_sour_Apr['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Apr['linkKey'] = ""
df_sour_Apr['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Apr['in_Amount'] = df_sour['Apr'] # <- change here
df_sour_Apr['in_PopulationServed'] = ""
df_sour_Apr['in_BenUse'] = "DCII"
df_sour_Apr['in_CustomerTypeCV'] = "DCII"
df_sour_Apr['in_ReportYearCV'] = df_sour['Year']
df_sour_Apr['in_TimeframeEnd'] = '04/30/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Apr['in_TimeframeStart'] = '04/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Apr['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Apr['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Apr['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Apr['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Apr['in_PODorPOUSite'] = "POD"
df_sour_Apr['in_SiteName'] = df_sour['Source Name']
df_sour_Apr['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Apr['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Apr))
df_sour_Apr.head(1)

In [ ]:
# Source Data - Monthly May
df_sour_May = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_May['in_VariableCV'] = "Withdrawal"
df_sour_May['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_May['linkKey'] = ""
df_sour_May['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_May['in_Amount'] = df_sour['May'] # <- change here
df_sour_May['in_PopulationServed'] = ""
df_sour_May['in_BenUse'] = "DCII"
df_sour_May['in_CustomerTypeCV'] = "DCII"
df_sour_May['in_ReportYearCV'] = df_sour['Year']
df_sour_May['in_TimeframeEnd'] = '05/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_May['in_TimeframeStart'] = '05/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_May['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_May['in_CoordinateMethodCV'] = "Representation Node"
df_sour_May['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_May['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_May['in_PODorPOUSite'] = "POD"
df_sour_May['in_SiteName'] = df_sour['Source Name']
df_sour_May['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_May['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_May))
df_sour_May.head(1)

In [ ]:
# Source Data - Monthly Jun
df_sour_Jun = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Jun['in_VariableCV'] = "Withdrawal"
df_sour_Jun['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Jun['linkKey'] = ""
df_sour_Jun['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Jun['in_Amount'] = df_sour['Jun'] # <- change here
df_sour_Jun['in_PopulationServed'] = ""
df_sour_Jun['in_BenUse'] = "DCII"
df_sour_Jun['in_CustomerTypeCV'] = "DCII"
df_sour_Jun['in_ReportYearCV'] = df_sour['Year']
df_sour_Jun['in_TimeframeEnd'] = '06/30/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Jun['in_TimeframeStart'] = '06/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Jun['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Jun['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Jun['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Jun['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Jun['in_PODorPOUSite'] = "POD"
df_sour_Jun['in_SiteName'] = df_sour['Source Name']
df_sour_Jun['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Jun['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Jun))
df_sour_Jun.head(1)

In [ ]:
# Source Data - Monthly Jul
df_sour_Jul = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Jul['in_VariableCV'] = "Withdrawal"
df_sour_Jul['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Jul['linkKey'] = ""
df_sour_Jul['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Jul['in_Amount'] = df_sour['Jul'] # <- change here
df_sour_Jul['in_PopulationServed'] = ""
df_sour_Jul['in_BenUse'] = "DCII"
df_sour_Jul['in_CustomerTypeCV'] = "DCII"
df_sour_Jul['in_ReportYearCV'] = df_sour['Year']
df_sour_Jul['in_TimeframeEnd'] = '07/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Jul['in_TimeframeStart'] = '07/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Jul['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Jul['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Jul['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Jul['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Jul['in_PODorPOUSite'] = "POD"
df_sour_Jul['in_SiteName'] = df_sour['Source Name']
df_sour_Jul['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Jul['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Jul))
df_sour_Jul.head(1)

In [ ]:
# Source Data - Monthly Aug
df_sour_Aug = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Aug['in_VariableCV'] = "Withdrawal"
df_sour_Aug['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Aug['linkKey'] = ""
df_sour_Aug['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Aug['in_Amount'] = df_sour['Aug'] # <- change here
df_sour_Aug['in_PopulationServed'] = ""
df_sour_Aug['in_BenUse'] = "DCII"
df_sour_Aug['in_CustomerTypeCV'] = "DCII"
df_sour_Aug['in_ReportYearCV'] = df_sour['Year']
df_sour_Aug['in_TimeframeEnd'] = '08/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Aug['in_TimeframeStart'] = '08/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Aug['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Aug['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Aug['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Aug['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Aug['in_PODorPOUSite'] = "POD"
df_sour_Aug['in_SiteName'] = df_sour['Source Name']
df_sour_Aug['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Aug['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Aug))
df_sour_Aug.head(1)

In [ ]:
# Source Data - Monthly Sep
df_sour_Sep = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Sep['in_VariableCV'] = "Withdrawal"
df_sour_Sep['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Sep['linkKey'] = ""
df_sour_Sep['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Sep['in_Amount'] = df_sour['Sep'] # <- change here
df_sour_Sep['in_PopulationServed'] = ""
df_sour_Sep['in_BenUse'] = "DCII"
df_sour_Sep['in_CustomerTypeCV'] = "DCII"
df_sour_Sep['in_ReportYearCV'] = df_sour['Year']
df_sour_Sep['in_TimeframeEnd'] = '09/30/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Sep['in_TimeframeStart'] = '09/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Sep['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Sep['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Sep['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Sep['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Sep['in_PODorPOUSite'] = "POD"
df_sour_Sep['in_SiteName'] = df_sour['Source Name']
df_sour_Sep['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Sep['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Sep))
df_sour_Sep.head(1)

In [ ]:
# Source Data - Monthly Oct
df_sour_Oct = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Oct['in_VariableCV'] = "Withdrawal"
df_sour_Oct['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Oct['linkKey'] = ""
df_sour_Oct['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Oct['in_Amount'] = df_sour['Oct'] # <- change here
df_sour_Oct['in_PopulationServed'] = ""
df_sour_Oct['in_BenUse'] = "DCII"
df_sour_Oct['in_CustomerTypeCV'] = "DCII"
df_sour_Oct['in_ReportYearCV'] = df_sour['Year']
df_sour_Oct['in_TimeframeEnd'] = '10/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Oct['in_TimeframeStart'] = '10/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Oct['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Oct['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Oct['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Oct['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Oct['in_PODorPOUSite'] = "POD"
df_sour_Oct['in_SiteName'] = df_sour['Source Name']
df_sour_Oct['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Oct['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Oct))
df_sour_Oct.head(1)

In [ ]:
# Source Data - Monthly Nov
df_sour_Nov = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Nov['in_VariableCV'] = "Withdrawal"
df_sour_Nov['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Nov['linkKey'] = ""
df_sour_Nov['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Nov['in_Amount'] = df_sour['Nov'] # <- change here
df_sour_Nov['in_PopulationServed'] = ""
df_sour_Nov['in_BenUse'] = "DCII"
df_sour_Nov['in_CustomerTypeCV'] = "DCII"
df_sour_Nov['in_ReportYearCV'] = df_sour['Year']
df_sour_Nov['in_TimeframeEnd'] = '11/30/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Nov['in_TimeframeStart'] = '11/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Nov['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Nov['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Nov['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Nov['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Nov['in_PODorPOUSite'] = "POD"
df_sour_Nov['in_SiteName'] = df_sour['Source Name']
df_sour_Nov['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Nov['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Nov))
df_sour_Nov.head(1)

In [ ]:
# Source Data - Monthly Dec
df_sour_Dec = pd.DataFrame(index=df_sour.index)

# Variable Info
df_sour_Dec['in_VariableCV'] = "Withdrawal"
df_sour_Dec['in_VariableSpecificCV'] = "Withdrawal_Monthly_DCII"

# SiteVariableAmounts_fact Info
df_sour_Dec['linkKey'] = ""
df_sour_Dec['in_CommunityWaterSupplySystem'] = df_sour['System Name']
df_sour_Dec['in_Amount'] = df_sour['Dec'] # <- change here
df_sour_Dec['in_PopulationServed'] = ""
df_sour_Dec['in_BenUse'] = "DCII"
df_sour_Dec['in_CustomerTypeCV'] = "DCII"
df_sour_Dec['in_ReportYearCV'] = df_sour['Year']
df_sour_Dec['in_TimeframeEnd'] = '12/31/' + df_sour['Year'].astype(str)  #<- change here
df_sour_Dec['in_TimeframeStart'] = '12/01/' + df_sour['Year'].astype(str) #<- change here

# Water Source Info
df_sour_Dec['in_WaterSourceTypeCV'] = df_sour['in_WaterSourceTypeCV']

# Site Info
df_sour_Dec['in_CoordinateMethodCV'] = "Representation Node"
df_sour_Dec['in_Latitude'] = df_sour['Lat NAD83'].astype(float)
df_sour_Dec['in_Longitude'] = df_sour['Lon NAD83'].astype(float)
df_sour_Dec['in_PODorPOUSite'] = "POD"
df_sour_Dec['in_SiteName'] = df_sour['Source Name']
df_sour_Dec['in_SiteNativeID'] = "POD" + df_sour['Source ID'].astype(str)
df_sour_Dec['in_SiteTypeCV'] = df_sour['Source Type']

print(len(df_sour_Dec))
df_sour_Dec.head(1)

In [ ]:
# Concatenate Source Data into one long dataframe.
frames = [df_sour_Ann, 
          df_sour_Jan, df_sour_Feb, df_sour_Mar, df_sour_Apr, df_sour_May, df_sour_Jun,
          df_sour_Jul, df_sour_Aug, df_sour_Sep, df_sour_Oct, df_sour_Nov, df_sour_Dec]
df_sour_out = pd.concat(frames)

df_sour_out = df_sour_out.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(df_sour_out))
df_sour_out.head(1)

# Concatenate System Data (POUs) with Source Data (PODs).

In [ ]:
# Concatenate Source Data into one long dataframe.
frames = [df_sys_out, df_sour_out]
dfout = pd.concat(frames)

dfout = dfout.replace(np.nan, "").drop_duplicates().reset_index(drop=True)
print(len(dfout))
dfout.head(1)

# Clean data

In [ ]:
# Convert History Year to string.

dfout['in_TimeframeEnd'] = pd.to_datetime(dfout['in_TimeframeEnd'], errors = 'coerce')
dfout['in_TimeframeEnd'] = pd.to_datetime(dfout["in_TimeframeEnd"].dt.strftime('%m/%d/%Y'))

dfout['in_TimeframeStart'] = pd.to_datetime(dfout['in_TimeframeStart'], errors = 'coerce')
dfout['in_TimeframeStart'] = pd.to_datetime(dfout["in_TimeframeStart"].dt.strftime('%m/%d/%Y'))

dfout.tail(1)

## WaDE Custom Elements (due to missing info)

In [ ]:
# updaing VariableSpecificCV to include a water source type

def updateVariableSpecificCV(valA, valB):
    valA = str(valA).strip()
    valB = str(valB).strip()
    outString = valA + "_" + valB
    return outString

dfout['in_VariableSpecificCV'] = dfout.apply(lambda row: updateVariableSpecificCV(row['in_VariableSpecificCV'], row['in_WaterSourceTypeCV']), axis=1)
dfout['in_VariableSpecificCV'].unique()

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEUT_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = dfout['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfout['in_WaterSourceNativeID'] = dfout.apply(lambda row: retrieveWaterSourceNativeID(row['in_WaterSourceTypeCV']), axis=1)
dfout['in_WaterSourceNativeID'].unique()

In [ ]:
# Converting in_PopulationServed to float value.
dfout['in_PopulationServed'] = dfout['in_PopulationServed'].replace(np.nan, 0, regex=True)
dfout['in_PopulationServed'] = dfout['in_PopulationServed'].replace('', 0, regex=True)
dfout['in_PopulationServed'] = dfout['in_PopulationServed'].astype(int)
dfout['in_PopulationServed']

## Shapefile Data
- For attaching gemetry to POU csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
ShapeFileInput = gpd.read_file('shapefile/CulinaryWaterServiceAreas.shp')
dfPoUshapetemp = pd.DataFrame(ShapeFileInput)
dfPoUshapetemp.head(3)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['WRID'].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Export Data

In [ ]:
#check datatype
print(len(dfout))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

In [ ]:
# Exporting output files.
dfout.to_csv('P_MasterUTSiteSpecific.csv', index=False)  # The output.
dfPoUshape.to_csv('P_utSSGeometry.csv', index=False) # The output geometry.